# Async Python

## A briefing on asynchronous python coding, essential in Agent engineering

Here is a masterful tutorial by you-know-who with exercises and comparisons.

https://chatgpt.com/share/680648b1-b0a0-8012-8449-4f90b540886c

This includes how to run async code from a python module.

### And now some examples:

In [26]:
# Let's define an async function

import asyncio
from cgitb import text

async def do_some_work(num):
    print("Starting work",num)
    await asyncio.sleep(1)
    print("Work complete",num)


In [28]:
# What will this do?

do_some_work(1)

<coroutine object do_some_work at 0x0000024BF6ADBD30>

In [29]:
# OK let's try that again!

await do_some_work(1)

Starting work 1
Work complete 1


In [31]:
# What's wrong with this?

async def do_a_lot_of_work(num):
    do_some_work(num)
    do_some_work(num+1)
    do_some_work(num+2)

await do_a_lot_of_work(1)

C:\Users\teds\AppData\Local\Temp\ipykernel_35728\11061074.py:4: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work(num)
C:\Users\teds\AppData\Local\Temp\ipykernel_35728\11061074.py:5: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work(num+1)
C:\Users\teds\AppData\Local\Temp\ipykernel_35728\11061074.py:6: RuntimeWarning: coroutine 'do_some_work' was never awaited
  do_some_work(num+2)


In [32]:
# Interesting warning! Let's fix it

async def do_a_lot_of_work(num):
    await do_some_work(num)
    await do_some_work(num+1)
    await do_some_work(num+2)

await do_a_lot_of_work(1)

Starting work 1
Work complete 1
Starting work 2
Work complete 2
Starting work 3
Work complete 3


In [44]:
# And now let's do it in parallel
# It's important to recognize that this is not "multi-threading" in the way that you may be used to
# The asyncio library is running on a single thread, but it's using a loop to switch between tasks while one is waiting

async def do_a_lot_of_work_in_parallel(num):
    await asyncio.gather(do_some_work(num), do_some_work(num+1), do_some_work(num+2))

await do_a_lot_of_work_in_parallel(10)

Starting work 10
Starting work 11
Starting work 12
Work complete 10
Work complete 11
Work complete 12


### Finally - try writing a python module that calls do_a_lot_of_work_in_parallel

See the link at the top; you'll need something like this in your module:

```python
if __name__ == "__main__":
    asyncio.run(do_a_lot_of_work_in_parallel())
```

In [45]:
async def do_a_lot_alot_of_work_in_parallel(num):
    await asyncio.gather(do_a_lot_of_work_in_parallel(num), do_a_lot_of_work_in_parallel(num+10), do_a_lot_of_work_in_parallel(num+20))

await do_a_lot_alot_of_work_in_parallel(10)

Starting work 10
Starting work 11
Starting work 12
Starting work 20
Starting work 21
Starting work 22
Starting work 30
Starting work 31
Starting work 32
Work complete 10
Work complete 12
Work complete 30
Work complete 32
Work complete 22
Work complete 31
Work complete 21
Work complete 11
Work complete 20


### Flow Diagram: Understanding the Execution Order

The following diagram shows how `do_a_lot_alot_of_work_in_parallel(10)` executes:

```mermaid
graph TD
    Start[Start: do_a_lot_alot_of_work_in_parallel] --> Gather1[asyncio.gather starts 3 groups]
    
    Gather1 --> Group1[Group 1: do_a_lot_of_work_in_parallel]
    Gather1 --> Group2[Group 2: do_a_lot_of_work_in_parallel]
    Gather1 --> Group3[Group 3: do_a_lot_of_work_in_parallel]
    
    Group1 --> G1Gather[asyncio.gather: tasks 10, 11, 12]
    Group2 --> G2Gather[asyncio.gather: tasks 20, 21, 22]
    Group3 --> G3Gather[asyncio.gather: tasks 30, 31, 32]
    
    G1Gather --> T10[Task 10: Start]
    G1Gather --> T11[Task 11: Start]
    G1Gather --> T12[Task 12: Start]
    
    G2Gather --> T20[Task 20: Start]
    G2Gather --> T21[Task 21: Start]
    G2Gather --> T22[Task 22: Start]
    
    G3Gather --> T30[Task 30: Start]
    G3Gather --> T31[Task 31: Start]
    G3Gather --> T32[Task 32: Start]
    
    T10 --> Sleep10[Sleep 1s]
    T11 --> Sleep11[Sleep 1s]
    T12 --> Sleep12[Sleep 1s]
    T20 --> Sleep20[Sleep 1s]
    T21 --> Sleep21[Sleep 1s]
    T22 --> Sleep22[Sleep 1s]
    T30 --> Sleep30[Sleep 1s]
    T31 --> Sleep31[Sleep 1s]
    T32 --> Sleep32[Sleep 1s]
    
    Sleep10 --> Complete10[Complete 10]
    Sleep11 --> Complete11[Complete 11]
    Sleep12 --> Complete12[Complete 12]
    Sleep20 --> Complete20[Complete 20]
    Sleep21 --> Complete21[Complete 21]
    Sleep22 --> Complete22[Complete 22]
    Sleep30 --> Complete30[Complete 30]
    Sleep31 --> Complete31[Complete 31]
    Sleep32 --> Complete32[Complete 32]
    
    Complete10 --> End[All Complete]
    Complete11 --> End
    Complete12 --> End
    Complete20 --> End
    Complete21 --> End
    Complete22 --> End
    Complete30 --> End
    Complete31 --> End
    Complete32 --> End
    
    style Start fill:#e1f5ff
    style Gather1 fill:#fff4e1
    style End fill:#e8f5e9
    style Sleep10 fill:#ffebee
    style Sleep11 fill:#ffebee
    style Sleep12 fill:#ffebee
    style Sleep20 fill:#ffebee
    style Sleep21 fill:#ffebee
    style Sleep22 fill:#ffebee
    style Sleep30 fill:#ffebee
    style Sleep31 fill:#ffebee
    style Sleep32 fill:#ffebee
```

**Key Points:**
1. **All tasks start immediately** - The event loop schedules all 9 tasks concurrently
2. **All tasks sleep simultaneously** - They all enter `asyncio.sleep(1)` at roughly the same time
3. **Completion order is non-deterministic** - Small timing variations cause different completion orders
4. **Single-threaded concurrency** - The event loop switches between tasks while they're waiting


### Timeline View: What Actually Happens

Here's a simplified timeline showing the execution flow:

```
Time →
│
├─ t=0ms:  Call do_a_lot_alot_of_work_in_parallel(10)
│          └─ asyncio.gather() starts 3 groups simultaneously
│
├─ t=0ms:  Group 1: do_a_lot_of_work_in_parallel(10)
│          ├─ asyncio.gather() starts tasks 10, 11, 12
│          ├─ Task 10: print("Starting work 10") ✓
│          ├─ Task 11: print("Starting work 11") ✓
│          └─ Task 12: print("Starting work 12") ✓
│
├─ t=0ms:  Group 2: do_a_lot_of_work_in_parallel(20)
│          ├─ asyncio.gather() starts tasks 20, 21, 22
│          ├─ Task 20: print("Starting work 20") ✓
│          ├─ Task 21: print("Starting work 21") ✓
│          └─ Task 22: print("Starting work 22") ✓
│
├─ t=0ms:  Group 3: do_a_lot_of_work_in_parallel(30)
│          ├─ asyncio.gather() starts tasks 30, 31, 32
│          ├─ Task 30: print("Starting work 30") ✓
│          ├─ Task 31: print("Starting work 31") ✓
│          └─ Task 32: print("Starting work 32") ✓
│
├─ t=0ms:  ALL 9 tasks enter asyncio.sleep(1) simultaneously
│          └─ Event loop suspends all tasks and waits
│
├─ t=1000ms: Tasks start waking up (order varies slightly)
│            ├─ Task 10 wakes: print("Work complete 10") ✓
│            ├─ Task 12 wakes: print("Work complete 12") ✓
│            ├─ Task 30 wakes: print("Work complete 30") ✓
│            ├─ Task 32 wakes: print("Work complete 32") ✓
│            ├─ Task 22 wakes: print("Work complete 22") ✓
│            ├─ Task 31 wakes: print("Work complete 31") ✓
│            ├─ Task 21 wakes: print("Work complete 21") ✓
│            ├─ Task 11 wakes: print("Work complete 11") ✓
│            └─ Task 20 wakes: print("Work complete 20") ✓
│
└─ t=1000ms: All tasks complete, function returns
```

**Why the completion order varies:**
- All tasks sleep for ~1000ms, but exact timing differs slightly
- The event loop schedules wake-ups based on when timers expire
- Small system delays can affect which task wakes first
- This demonstrates true concurrency: tasks don't wait for each other


### Why Task 10, then 12, then 30? Understanding the "Random" Order

The completion order (10, 12, 30, 32, 22, 31, 21, 11, 20) seems random, but here's what's actually happening:

**The Key Insight: All tasks are racing to finish!**

Think of it like 9 runners starting a race at the exact same moment:
- They all start running at t=0ms
- They all need to run for exactly 1 second
- But tiny differences in when they actually cross the finish line create the order

**What's happening under the hood:**

```
All 9 tasks call asyncio.sleep(1) at nearly the same time:

Task 10: sleep(1) → timer set for 1000.0001ms
Task 11: sleep(1) → timer set for 1000.0003ms  (slightly later!)
Task 12: sleep(1) → timer set for 1000.0002ms
Task 20: sleep(1) → timer set for 1000.0005ms  (even later!)
Task 21: sleep(1) → timer set for 1000.0004ms
Task 22: sleep(1) → timer set for 1000.0006ms
Task 30: sleep(1) → timer set for 1000.0000ms  (earliest!)
Task 31: sleep(1) → timer set for 1000.0007ms
Task 32: sleep(1) → timer set for 1000.0001ms  (tied with 10!)

When timers expire:
- Task 30 wakes first (1000.0000ms) → prints "Complete 30"
- Task 10 wakes next (1000.0001ms) → prints "Complete 10"  
- Task 32 wakes next (1000.0001ms) → prints "Complete 32"
- Task 12 wakes next (1000.0002ms) → prints "Complete 12"
- ... and so on
```

**Why these tiny differences exist:**
1. **Event loop scheduling** - Tasks are scheduled in the order they're created, but there's a tiny delay between each
2. **System timer precision** - Your OS's timer might have microsecond-level variations
3. **Event loop overhead** - The loop itself takes tiny amounts of time to process each task
4. **Python interpreter** - Small variations in how Python schedules coroutines

**Important:** This order is **non-deterministic** - if you run it again, you might get a different order! That's the nature of concurrent execution.


In [ ]:
# Let's see the actual timing differences!

import asyncio
import time

async def do_some_work_with_timing(num):
    start_time = time.time()
    print(f"Starting work {num} at {start_time:.6f}")
    await asyncio.sleep(1)
    end_time = time.time()
    elapsed = end_time - start_time
    print(f"Work complete {num} at {end_time:.6f} (elapsed: {elapsed:.6f}s)")

async def demonstrate_timing():
    start = time.time()
    print(f"All tasks starting at: {start:.6f}\n")
    await asyncio.gather(
        do_some_work_with_timing(10),
        do_some_work_with_timing(11),
        do_some_work_with_timing(12)
    )
    print(f"\nAll tasks finished at: {time.time():.6f}")

await demonstrate_timing()


### Visual: Why 10, 12, 30 (and not 10, 11, 12)?

Here's a side-by-side view showing what's happening:

```
Task Execution Timeline (exaggerated for clarity):

Task 10: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 1st
Task 11: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 8th
Task 12: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 2nd
Task 20: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 9th
Task 21: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 7th
Task 22: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 5th
Task 30: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 3rd
Task 31: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 6th
Task 32: |====START====|----SLEEP----|====COMPLETE====|  → Finishes 4th
         ↑            ↑              ↑
       All start   All sleep      Wake up order
       together    together       varies slightly
```

**The Critical Point:**

Even though tasks 10, 11, 12 are in the same `asyncio.gather()` call, they don't complete in order (10, 11, 12). Instead, you see 10, 12, 30, 32, 22, 31, 21, 11, 20.

**Why?** Because:
- All 9 tasks are **truly concurrent** - they're all part of the same event loop
- The event loop doesn't care which `gather()` they came from
- It just wakes them up in the order their timers expire
- Task 10's timer expires slightly before Task 11's timer
- Task 30's timer expires even earlier than Task 10's!

**This is the beauty of async:** Tasks from different groups can interleave their completion, showing true concurrency!
